# Numpy implementation of the structure function

This notebooks shows a numpy implemetation of the structure function as describe e.g. in https://www.aanda.org/articles/aa/full/2007/05/aa6283-06/aa6283-06.right.html. The overall performance is a factor of ~30 better than the pure Python version.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.testing import assert_allclose
from collections import Counter

In [2]:
def StructureFunction(_time, _flux):
    # define some variables
    delta_tau, delta_flux, Tau, SF = [],[],[],[]
    NN = len(_time)
    # Loop over all (i,j) pairs
    for i in range(NN) :
        for j in range(i+1,NN):
            delta_tau.append(_time[j]-_time[i])
            delta_flux.append((_flux[j]-_flux[i])**2)
    
    # Count # of each element
    NCount = dict(Counter(delta_tau))
    for counter in NCount :
        Tau.append(counter)

    #Compute SF
    for i in NCount:
        k     = 0
        summe = 0.0
        for j in delta_tau:
            if j == i :
                summe +=delta_flux[k]
            k +=1
        SF.append(summe/NCount[i])
    return Tau, SF

In [3]:
def structure_function(time, flux):
    i, j = np.indices((len(time), len(time)))
    selection = (i < j)
    delta_time = (time - time[:, np.newaxis])[selection]
    delta_flux = ((flux - flux[:, np.newaxis]) ** 2)[selection]
    
    tau, n_tau = np.unique(delta_time, return_counts=True)
    
    weights = (delta_time == tau[:, np.newaxis])   
    
    # use matrix vector multiplication to compute the sum 
    sf = 1. / n_tau * np.dot(weights, delta_flux)
    return tau, sf

In [4]:
np.random.seed(0)
N = 100
time = np.random.randint(0, 100, N)
flux = np.sin(0.1 * np.pi * time)

In [5]:
%%timeit
tau_ref, sf_ref = StructureFunction(time, flux)

67.9 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
tau, sf = structure_function(time, flux)

2.78 ms ± 34 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
#assert the the results are the same, 
tau, sf = structure_function(time, flux)
tau_ref, sf_ref = StructureFunction(time, flux)
idx = np.argsort(tau_ref)
assert_allclose(tau, np.array(tau_ref)[idx])
assert_allclose(sf, np.array(sf_ref)[idx])